In [ ]:
import requests
from bs4 import BeautifulSoup
import base64
import os
from PIL import Image
import io
import json

API_KEY = 'Hqr5V8CxDq3RgaEq9WD70j7JtpbrXx1s'

URL_SCRAPING = 'https://intern.aiaxuropenings.com/scrape/272e0108-52dd-47ce-9919-b35da203104b'
URL_MODEL = 'https://intern.aiaxuropenings.com/v1/chat/completions'
URL_SUBMIT = 'https://intern.aiaxuropenings.com/api/submit-response'

dados_imagem = 'imagem_scrapeada.jpg'

# faz o scraping da imagem no site
def baixar_imagem():
    response = requests.get(URL_SCRAPING)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    # busca a tag da imagem
    img_tag = soup.find('img', alt='Imagem aleatória')

    if not img_tag:
        raise Exception('Nenhuma tag <img> encontrada na página.')

    # pega o src da imagem e faz o tratamento para base64
    dados_img = img_tag['src']
    if not dados_img.startswith('data:image'):
        raise Exception('O src da imagem não está em formato base64 embutido.')

    header, encoded = dados_img.split(',', 1)
    dados_limpo = base64.b64decode(encoded)

    # salva a imagem localmente
    with open(dados_imagem, 'wb') as f:
        f.write(dados_limpo)

    print(f'Imagem salva como {dados_imagem}')

# faz a requisição para o modelo e retorna o resultado
def enviar_para_modelo():
    print('Enviando imagem para o modelo...')

    # abre a imagem e faz compressão agressiva para reduzir tokens
    with Image.open(dados_imagem) as img:
        img = img.convert('RGB')
        img.thumbnail((128, 128))  # Reduz muito a resolução para caber nos tokens

        buffer = io.BytesIO()
        img.save(buffer, format='JPEG', quality=20, optimize=True)  # Qualidade muito baixa
        image_bytes = buffer.getvalue()

    # codifica a imagem em base64
    image_base64 = base64.b64encode(image_bytes).decode('utf-8')

    # monta o header da requisição
    headers = {
        'Authorization': f'Bearer {API_KEY}',
        'Content-Type': 'application/json'
    }

    # monta o payload no formato correto, com a imagem em base64 embutida no prompt
    payload = {
        'model': 'microsoft-florence-2-large',
        'messages': [
            {
                'role': 'user',
                'content': f'<DETAILED_CAPTION>data:image/jpeg;base64,{image_base64}</DETAILED_CAPTION>'
            }
        ],
        'temperature': 0.2,
        'stream': False
    }

    # faz a requisição POST para o modelo
    response = requests.post(
        URL_MODEL,
        headers=headers,
        json=payload
    )

    # trata possíveis erros na requisição
    if response.status_code != 200:
        print(f'Erro na requisição ao modelo: {response.status_code}')
        print(response.text)
        raise Exception('Erro na API do modelo.')

    # pega o resultado da requisição
    result = response.json()
    print('Resposta do modelo recebida:')
    print(result)
    return result

# faz a requisição para o endpoint de submissão
def submeter_resposta(result):
    print('Submetendo resposta...')

    # monta o header da requisição
    headers = {
        'Authorization': f'Bearer {API_KEY}',
    }

    # faz a requisição POST para enviar a resposta
    response = requests.post(
        URL_SUBMIT,
        headers=headers,
        json=result
    )

    # trata possíveis erros na submissão
    if response.status_code == 200:
        print('Resposta submetida com sucesso!')
        print(response.json())
    else:
        print(f'Erro na submissão: {response.status_code}')
        print(response.text)
        raise Exception('Erro na submissão.')



In [30]:
baixar_imagem()
resultado = enviar_para_modelo()
submeter_resposta(resultado)

Imagem salva como imagem_scrapeada.jpg
Enviando imagem para o modelo...
Resposta do modelo recebida:
{'choices': [{'finish_reason': 'length', 'index': 0, 'logprobs': None, 'message': {'content': "It appears that the image you've provided is a JPEG file with a base64 encoded string. The base64 string is:\n\n```\n/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDACgcHiMeGSgjISMtKygwPGRBPDc3PHtYXUlkkYCZlo+AjIqgtObDoKrarYqMyP/L2u71////m8H////6/+b9//j/2wBDASstLTw1PHZBQXb4pYyl+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj4+Pj